#Оценка качества изображений

**Постановка задачи**: Фотографии загружаемые поставщиками WB имеют разное качество: На одних может быть сложный фон, на каких-то фотографиях часть объекта не попала в кадр и.т.п. Для последующей работы с такими данными, например при использовании алгоритмов поиска по фото надо знать типs дефектов/овособенностей которые присутствуют на изображении.

<img src ="https://ml.gan4x4.ru/wb/quality/content/samples.png" width="800">


Всего 6 типов особенностей:  

* untidy,
* angle-composition,
* background,
* crop,
* text,
* multiple-objects

и один класс для изображений без дефектов
* good-image.

При этом изображение может содержать несколько видов дефектов.


Задача:

Требуется создать модель которая будет определять список дефектов для для изображения.

# Данные

По [ссылке](https://ml.gan4x4.ru/wb/quality/5000/student_5000.zip) доступен архив содержащий 5000 изображений и разметку.

Оригинальные изображения имели размер 900x1200 в датасете их разрешение уменьшено вдвое. Кроме изображений в архиве находиться csv файл c разметкой.
В первой колонке имя файла с изображением (без расширения), в остальных колонках названия классов к которым относиться изображение:

```
  18715,text,multiple-objects,,
  5259,text,background,,
  8932,background,,,
  ...

```

# Порядок выполнения задания

Задание рекомендуется выполнять по шагам:

1. Познакомьтесь с данными
2. Выберите метрику для оценки результата
3. Проведите анализ состояния вопроса, изучите существующие модели которые можно использовать для решения задачи
4. Проведите EDA, опишите особенности данных и проблемы которые они могут за собой повлечь
5. Подготовьте данные для обучения
6. Выберите baseline модель, оцените качество её работы на данном датасете.
7. Попробуйте улучшить значение метрики используя другую модель. Возможно обучив/дообучив ее.
8. Оцените быстродействие выбранной модели
9. Дайте оценку полученному результату.


**Важно!**

Блокнот должен содержать весь необходимый код для запуска финальной модели. Если для запуска требуется подгрузка весов, все ссылки длжны работать не только в вашем аккаунте но и в аккаунте преподавателя.

## Анализ данных (EDA)

In [1]:
!wget https://ml.gan4x4.ru/wb/quality/5000/student_5000.zip
!unzip student_5000.zip

--2024-06-21 06:35:11--  https://ml.gan4x4.ru/wb/quality/5000/student_5000.zip
Resolving ml.gan4x4.ru (ml.gan4x4.ru)... 212.24.105.216
Connecting to ml.gan4x4.ru (ml.gan4x4.ru)|212.24.105.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114875224 (110M) [application/zip]
Saving to: ‘student_5000.zip’

student_5000.zip    100%[===================>] 109.55M  6.89MB/s    in 13s     

2024-06-21 06:35:25 (8.13 MB/s) - ‘student_5000.zip’ saved [114875224/114875224]

Archive:  student_5000.zip
   creating: 5000/
  inflating: 5000/5000.csv           
   creating: 5000/images/
  inflating: 5000/images/18715.jpg   
  inflating: 5000/images/6070.jpg    
  inflating: 5000/images/9138.jpg    
  inflating: 5000/images/8672.jpg    
  inflating: 5000/images/26485.jpg   
  inflating: 5000/images/25605.jpg   
  inflating: 5000/images/18417.jpg   
  inflating: 5000/images/10337.jpg   
  inflating: 5000/images/7571.jpg    
  inflating: 5000/images/22709.jpg   
  inflating: 50

In [1]:
# ------------------------------------- #
#               Libraries               #
# ------------------------------------- #

import os, re, gc
import sys,time,random
# from numba import jit , cuda
import numpy as np
import pandas as pd
import shutil
import pathlib
import pickle
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import PIL
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader , Dataset
from torch.optim import AdamW
import torchvision
from torchvision import models
from torchvision import transforms as T

import timm

import pytorch_lightning as pl
from pytorch_lightning import LightningModule,Trainer
from pytorch_lightning.loggers import TensorBoardLogger


print("Done!")

Done!


In [5]:
# -------------------------------------- #
#     training dirs, model paths,etc     #
# -------------------------------------- #

input_dir = "./data"
train_dir = "./data/5000/images"
train_csv_path = "./data/5000/5000.csv"

In [3]:
# ---------------------------------------------------- #
#    Configuration class contains training configs     #
#                  &  model configs                    #
# ---------------------------------------------------- #

class Config:
        BATCH_SIZE = 32
        KFOLD = 5
        FOLD = 0
        SEED = 21
        LEARNING_RATE = 8e-6
        TYPE  = "Multi-label Image Classification"
        DATA_SOURCE = "multi-label-image-classification-dataset"
        MODEL_NAME = 'resnetv2_50'
        CRITERION_ = "Binary Cross Entropy"
        OPTIMIZER_ = "AdamW"
        DATA_TYPE = 'image'
        DATA_FORMAT = '.jpg'
        IMG_SIZE = (384,384)
        EPOCHS = 10
        
        def __init__(self):
            print("configuration set!")
        
        def check_cuda(self):
            print("Scanning for CUDA")
            if torch.cuda.is_available():
                print("GPU is available , training will be accelerated! : )\n")
            else:
                print("NO GPUs found : / \n")
        
        def seed_everything(self):
            print("Seeding...")
            np.random.seed(self.SEED)
            random.seed(self.SEED)
            os.environ['PYTHONHASHSEED'] = str(self.SEED)
            torch.manual_seed(self.SEED)
            torch.cuda.manual_seed(self.SEED)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            print("Seeded everything!")

CFG = Config()
CFG.seed_everything()

configuration set!
Seeding...
Seeded everything!


In [6]:
train_df = pd.read_csv(train_csv_path)
train_df

,18715,text,multiple-objects,Unnamed: 3,Unnamed: 4
0,5259,text,background,NaN,NaN
1,8932,background,NaN,NaN,NaN
2,6070,good-image,NaN,NaN,NaN
3,13889,good-image,NaN,NaN,NaN
4,24891,good-image,NaN,NaN,NaN
...,...,...,...,...,...
4994,12027,text,NaN,NaN,NaN
4995,23246,good-image,NaN,NaN,NaN
4996,1534,good-image,NaN,NaN,NaN
4997,10381,background,NaN,NaN,NaN


In [20]:
from PIL import Image
from pathlib import Path

DATA = Path('./5000/images/')

images = []
# iterate over files in
# that directory
for file in DATA.glob('*'):
    image = Image.open(file)
    images += image


TypeError: 'JpegImageFile' object is not iterable

In [12]:
# Познакомьтесь с данными
import pandas as pd

labels = pd.read_csv('5000/5000.csv', header=None, index_col=0)
labels.

,1,2,3,4
0,,,,
18715,text,multiple-objects,NaN,NaN
5259,text,background,NaN,NaN
8932,background,NaN,NaN,NaN
6070,good-image,NaN,NaN,NaN
13889,good-image,NaN,NaN,NaN
...,...,...,...,...
12027,text,NaN,NaN,NaN
23246,good-image,NaN,NaN,NaN
1534,good-image,NaN,NaN,NaN


# Метрики

In [ ]:
# Put your code here

## Baseline

In [ ]:
# Put your code here

## Решение

In [ ]:
# Put your code here

## Оценка результата

In [ ]:
# Put your code here

## Вывод

...

# Тестовый блок для проверки

Поместите сюда весь необходимый код для тестирования вашей модели на новых данных. Убедитесь что

- Импортируются все библиотеки и классы
- Подгружабтся веса с внешних ресурсов
- Происходит рассчет метрик
...

In [ ]:
# Put your code here